In [1]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
# using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
# using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
# using JuliaDB
# using Wavelets,WaveletsExt
# using LinearAlgebra
# import Pkg; Pkg.add("ShiftedArrays")
# using ShiftedArrays
# include("utils.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4746327218440964993\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10398097852411983091\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4746327218440964993\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [3]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

129177

In [4]:
start_date = DateTime(2021, 5, 1)
end_date = DateTime(2021, 5, 21)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

28801

In [ ]:
ar = cc
ar_len = length(ar)
plot_box = Observable{Any}(dom"div"())


window_size = 60 * 8 * 1

num_windows = Int64(trunc(ar_len/window_size*4))

window_no_slider = slider(1:num_windows; label="start", default=7)

window_start = Interact.@map max(1, Int(&window_no_slider * window_size / 4))
window_end = Interact.@map min(ar_len, &window_start+window_size-1)

rng = Interact.@map &window_start: &window_end

put = Interact.@map ut[&rng]

# wdfo = Interact.@map dfo[(dfo.ut .>= first(&put)) .& (dfo.ut .<= last(&put)), :]

function draw_it_all(rng)
    put = ut[rng]
    wtdo = dfo[(dfo.ut .>= first(put)) .& (dfo.ut .<= last(put)), :]
    
    buys = wtdo[wtdo.reason .== :enter,:]
    losses = wtdo[(wtdo.reason .== :exit) && (wtdo.pnl .< 0),:]
    profits = wtdo[(wtdo.reason .== :exit) && (wtdo.pnl .> 0),:]
#     toolongs = dfo[(dfo.reason .== :toolong),:]
    
    f1 = plot(ut[rng], cc[rng], size=(1200,300),legend=nothing)
    plot!(put,ema50[rng])
#     plot!(put,ema100[rng])
#     plot!(put,ema150[rng])    
    
    plot!(put,ema50[rng]*0.995, fillrange=ema50[rng]*1.005, fillalpha=0.1)
    
    nrow(buys) > 0 && vline!(buys.ut, color=:blue)
    nrow(profits) > 0 && vline!(profits.ut, color=:green)
    nrow(losses) > 0 && vline!(losses.ut, color=:red)
    
    f2 = plot(put,rsi14[rng], linewidth=1, color=:blue)
    plot!(put,rsi26[rng], linewidth=1, color=:red)
    #     plot!(ut,rsi26)\n",
    hline!([30,70])
    plt = plot(f1, f2, layout=@layout[a{0.6h}; b{0.25h};], legend=nothing, size=(1200,400))
    dom"div"(vbox(plt,wtdo))
end

map!(draw_it_all, plot_box, rng)
ui = dom"div"(window_no_slider,plot_box)


In [ ]:
# x = [3, 1, 2];
# df = DataFrame(x=x)
df
r = last(df)
r.x = 22
df

In [15]:
function backtest(e1, f1, e1ch1, e1ch2, far_1_dx2, sl)
    capital = 10000.
    risk_percent = 0.02
    qfee = 0.001
    

    dfo = DataFrame(ut = Int64[], 
            order = Symbol[],
            oix = Int64[],
            reason = Symbol[],
            price = Float64[],
            target = Float64[],
            stop = Float64[],
            qty = Float64[],
            pnl = Float64[],
            capital = Float64[]
        )

    push!(dfo, (ut = ut[1], oix =0, order = :long, reason = :toolong, price = 0, stop = 0, target = 0,
                qty = 0, pnl = 0, capital = capital))


    in_long_pos = false 
    qlsellsig = 1.01
    qstop = 0.01
    in_i = 0
    for i in 200:length(cc)

        ord = last(dfo)
        if !in_long_pos
            if true && 
#                 far_1_dx2[i-1] < 0 && far_1_dx2[i] > 0
                cc[i] < f1[i] * (1 - e1ch1/100) &&
                cc[i] >  f1[i] * (1 - e1ch2/100) 
#                 cc[i] > pema_corr[i] * (1 - channel/100) && 
#                 cc[i-1] < pema_corr[i-1] * (1 - channel/100) 
#                     cc[i] < pema_corr[i] &&
#                     cc[i-1] > pema_corr[i-1] 


                stop_price = cc[i] * (1 - sl/100) # * (1-qfee)

                risk_size = capital * risk_percent

                buy_price = cc[i]*(1+qfee)

                risk_price_range = (buy_price - stop_price)
                buyq = risk_size / risk_price_range

                target = cc[i] * 1.01  # 2*risk_price_range

                push!(dfo, (ut = ut[i], oix=i, order = :long, reason = :enter, price = buy_price, qty = buyq,
                        target = target, stop = stop_price, pnl = 0, capital = capital))

                in_long_pos = true
            end
        else
#             last_min = minimum(cc[i-movesl:i-3])
            
            if cc[i] > f1[i]
                ord.stop = f1[i] * (1+e1ch2/100)
            end
#             if i - ord.oix > per_expire && cc[i] > pema_corr[i] * (1+channel/100)
# #                 cc[i] > pema_corr[i]
                
#                 ord.stop = pema_corr[i] * (1+channel/100)
#     #             println("TARGET:\t$profit_loss, ")

#             end
# #             ord = last(dfo)
            
            if cc[i] <= ord.stop # SL
                sell_price = cc[i]*(1-qfee)
                profit_loss = ord.qty * (sell_price - ord.price)
                capital = capital + profit_loss

                push!(dfo, (ut = ut[i], oix=i, order = :long, reason = :exit, price = sell_price, stop = 0, 
                        target = 0, qty = ord.qty, pnl = profit_loss, capital = capital))

                in_long_pos = false
            end
           

        end
    #     println(capital)

    end
    CSV.write("trades.csv", dfo)
    dfo
end

backtest (generic function with 1 method)

In [16]:
function smooth(ar,ema_1_sm_2_q, lag)
    nar = circshift(ar,lag)
    dar = ar - nar
    xar = ar + dar*ema_1_sm_2_q
end
function smooth2(ar, ema_1_sm_2_q, lag1, lag2)
    nar = smooth(ar,ema_1_sm_2_q,lag1)    
    xar = smooth(nar,ema_1_sm_2_q,lag2)
end
function der(ar, lag)
    nar = circshift(ar,lag)
    dar = ar - nar
end
function der2(ar, lag1, lag2)
    nar = der(ar,lag1)    
    xar = der(nar,lag2)
end
ar_len = length(cc)
plot_box = Observable{Any}(dom"div"())
window_size = 60 * 8 * 1
num_windows = Int64(trunc(ar_len/window_size*4))
window_no_slider = slider(1:num_windows; label="start", default=7)
window_start = Interact.@map max(1, Int(&window_no_slider * window_size / 4))
window_end = Interact.@map min(ar_len, &window_start+window_size-1)
rng = Interact.@map &window_start: &window_end
put = Interact.@map ut[&rng]
ema_1_slider = slider(10:5:150; label="ema", default=50)
ema_1_slider[]=100
ema_1 = Interact.@map ema(cc, n=&ema_1_slider)
ema_1_dx_1_lag_slider = slider(0:100; label="e1dx1", default=14)
ema_1_dx_1_lag_slider[] = 10
ema_1_dx_2_lag_slider = slider(0:100; label="e1dx2", default=14)
ema_1_dx_2_lag_slider[] = 10

ema_1_ch_1_slider = slider(-5:0.05:5; label="ech1", default=0.5)
ema_1_ch_1_slider[]=0.8
ema_1_ch_2_slider = slider(-5:0.05:5; label="ech2", default=0.5)
ema_1_ch_2_slider[]=2.5
ema_1_sm_2_q = slider(-5:0.05:5; label="q1", default=0.5)
ema_1_sm_2_q[]=-1.75

far_1 = Interact.@map smooth2(&ema_1,&ema_1_sm_2_q, &ema_1_dx_1_lag_slider,&ema_1_dx_2_lag_slider )
# far_1_der_2 = der2(&far_1,1,1)
sl_slider = slider(0:0.05:5; label="sl", default=1)
sl_slider[]=2

far_1_dx2 = Interact.@map der2(&far_1,1,1)

# expire_slider = slider(1:200; label="exp", default=1)
# expire_slider[]=33
# movesl_slider = slider(1:50; label="movsl", default=1)
# movesl_slider[]=4

dfo = Interact.@map backtest(&ema_1, &far_1, &ema_1_ch_1_slider, &ema_1_ch_2_slider, &far_1_dx2, &sl_slider)

wdfo = Interact.@map filter(row -> (row.ut >= ut[&window_start] && row.ut <= ut[&window_end]), &dfo, view=false)
last2dfo = Interact.@map last(&dfo, 2)
# &dfo[(&dfo.ut .>= &window_start) .& (&dfo.ut .<= &window_end), :]

function draw_it_all(rng, ema_1, far_1, ema_1_ch_1, ema_1_ch_2, wdfo)
    put = ut[rng]   
    
    buys = wdfo[wdfo.reason .== :enter,:]
    losses = wdfo[(wdfo.reason .== :exit) .& (wdfo.pnl .< 0),:]
    profits = wdfo[(wdfo.reason .== :exit) .& (wdfo.pnl .> 0),:]
    
    f1 = plot(ut[rng], cc[rng], size=(1200,500))
    plot!(put,ema_1[rng], label="ema_1",lw=0.8)
    
    plot!(put,far_1[rng]*(1-ema_1_ch_1/100), fillrange=far_1[rng]*(1+ema_1_ch_1/100),
        lw=0,fillalpha=0.2, label="ch1")
    
    plot!(put,far_1[rng]*(1-ema_1_ch_2/100), fillrange=far_1[rng]*(1+ema_1_ch_2/100),
        lw=0,fillalpha=0.2, label="ch2")
# #     f1 = plot!(put,pema_lag[rng],label="lag ema",lw=1)
    f1 = plot!(put,far_1[rng], label="far_1",lw=1)
    
    nrow(buys) > 0 && vline!(buys.ut,lw=0.5, color=:blue)
    nrow(profits) > 0 && vline!(profits.ut,lw=0.5, color=:green)
    nrow(losses) > 0 && vline!(losses.ut,lw=0.5, color=:red)
    
    dom"div"(vbox(f1))
end

map!(draw_it_all, plot_box, rng, ema_1, far_1, ema_1_ch_1_slider, ema_1_ch_2_slider, wdfo)
ui = dom"div"(hbox(
            vbox(ema_1_slider,ema_1_dx_1_lag_slider,ema_1_dx_2_lag_slider),
            vbox(ema_1_ch_1_slider,ema_1_ch_2_slider,ema_1_sm_2_q),
            sl_slider
        ),
        plot_box,window_no_slider,last2dfo)

(div
  (div { style=Dict("display" => "flex", "flex-direction" => "row") }
    (div { style=Dict("display" => "flex", "flex-direction" => "column") }
      Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ema"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 29, :default => 50, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
19, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000010b55e540, Task (runnable) @0x000000010b55e540), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"10\",\"15\",\"20\",\"25\",\"30\",\"35\",\"40\",\"45\",\"50\",\"55\",\"60\",\"65\",\"70\",\"75\",\"80\",\"85\",\"90\",\"95\",\

In [9]:
nrow(dfo[][(dfo[].reason .== :exit) .& (dfo[].pnl .< 0),:]),
    nrow(dfo[][(dfo[].reason .== :exit) .& (dfo[].pnl .> 0),:])

(22, 0)

In [ ]:

dfo=0

In [ ]:
wave_slider[]

In [ ]:
i = 1000
i - max(1,i - 512)

In [ ]:
wt = wavelet(WT.Coiflet{4}(), WT.Filter, WT.Periodic)

In [ ]:
2^8

In [ ]:
length(ar)